<a href="https://colab.research.google.com/github/hamzamusht550/Amazon-Clone-Frontend/blob/main/MKV_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

drive.mount('/content/drive')
auth.authenticate_user()

!sudo apt install software-properties-common -y
!sudo add-apt-repository ppa:wahibre/mtn -y
!sudo add-apt-repository ppa:savoury1/ffmpeg4 -y
!apt-get install -y mediainfo ffmpeg mtn mkvtoolnix aria2 yt-dlp
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

#mtn
!sudo apt install mtn -y

#ffmpeg libfdk-aac with apt requirements
# !apt-get install -y autoconf automake build-essential cmake git-core libass-dev libfreetype6-dev libsdl2-dev libtool libva-dev libvdpau-dev libvorbis-dev libxcb1-dev libxcb-shm0-dev libxcb-xfixes0-dev pkg-config texinfo wget zlib1g-dev libmp3lame-dev libopus-dev libvpx-dev libx264-dev libx265-dev libfdk-aac-dev nasm
# !wget https://github.com/hamzamusht550/FFmpeg/releases/download/vN-118392-g0113e30806/ffmpeg_static.zip
# !7z e ffmpeg_static.zip
# !cp ffmpeg ffprobe ffplay /bin
# !rm ffmpeg ffprobe ffplay ffmpeg_static.zip

!sudo apt update && sudo apt upgrade -y
clear_output()

In [ ]:

#@title MKV Tools

# MKV = "https://mixology-stream-de9199b81403.herokuapp.com/41759/Pushpa+2+%3A+The+Rule+Reloaded+Version+%282025%29+720p+NF+WEB-DL+x.mkv?hash=AgAD9R" #@param {type: "string"}
MKV = "/content/drive/MyDrive/My Instant Death Ability/S01E01-Instant Death Ability [300A69A0].mkv" #@param {type: "string"}
SELECT_ACTION = "TRIM" #@param ["MEDIA_INFO", "TRIM", "SCREENSHOT", "THUMBNAIL"]
#@markdown ***Start 00:12:00 / End 00:13:00***
trim_start = "00:11:00"  #@param {type: "string"}
trim_end = "00:12:00"  #@param {type: "string"}
#@markdown ***Start 00:12:00 / End 00:13:00***
Screenshots_No = 3  #@param {type: "number"}
Screenshots_Custom = "-vf \"select='not(mod(n,FRAME_INTERVAL))*gt(mean_luma,50)'\" -vsync vfr"  #@param {type: "string"}
Thumb_Rows = 5  #@param {type: "number"}
Thumb_Cols = 2  #@param {type: "number"}
Thumb_Custom = ""  #@param {type: "string"}

import ffmpeg
import os
from PIL import Image
from IPython.display import display


def getDurationSec(file_path):
    probe = ffmpeg.probe(file_path)
    duration = float(probe['format']['duration'])
    duration_seconds = int(duration)
    return duration_seconds

MKV_File = ''
MKV_Name = ''
if os.path.exists(MKV):
    MKV = os.path.abspath(MKV)
    MKV_File = os.path.split(MKV)[1]
    MKV_Name = os.path.splitext(MKV_File)[0]

match SELECT_ACTION:
    case "MEDIA_INFO":
        # !ffprobe {MKV}
        !mediainfo {MKV}
        !du -h "{MKV}"
        !du -BM "{MKV}"

    case "TRIM":
        if trim_start:
            if len(trim_start) == len(trim_end) == 8:
                strTrim = f"-ss {trim_start} -to {trim_end}"
                !ffmpeg -i "{MKV}" {strTrim} -map 0:v:1 -map 0:a -map 0:s? -c copy -y "trim_{MKV_Name}.mkv"
                !du -h "trim_{MKV_Name}.mkv"
                !du -BM "trim_{MKV_Name}.mkv"
            else:
                print("Invalid Trim.")
        else:
            print("trim_start Shouldn't be Empty.")

    case "SCREENSHOT":
        if Screenshots_No != 551:
            # "screenshot%03d_{MKV_Name}.jpg"
            if Screenshots_Custom:
                !ffmpeg -hide_banner -loglevel error -i "{MKV}" {Screenshots_Custom} -frames:v {Screenshots_No}  "screenshot%03d_{MKV_Name}.jpg"
            else:
                !ffmpeg -hide_banner -loglevel error -i "{MKV}" -vf "thumbnail" -frames:v {Screenshots_No}  "screenshot%03d_{MKV_Name}.jpg"
            for i in range(Screenshots_No):
                img = f"screenshot{i+1:03}_{MKV_Name}.jpg" if os.path.exists(MKV) else f"screenshot{i+1:03}_.jpg"
                print(f"Screenshot({i+1}):\n{img}")
                display(Image.open(img))
        else:
            print("Minimum 2 Screenshots")
    case "THUMBNAIL":
        if Thumb_Custom:
            !mtn Thumb_Custom -o "thumb_{MKV_Name}.jpg" "{MKV}"
        else:
            !mtn -w 600 -r {Thumb_Rows} -c {Thumb_Cols} -D 12 -o "thumb_{MKV_Name}.jpg" "{MKV}"

In [ ]:

#@title MKV Merger
# green #00FF02

MKV_FILES = "/content/drive/MyDrive/My Instant Death Ability/S01E01-Instant Death Ability [300A69A0].mkv"  #@param {type: "string"}
Metadata = "@Anime_Closx"  #@param {type: "string"}
LANGUAGE = "All"  #@param ["All", "Custom", "Hindi", "English", "Japanese", "Tamil", "Telugu", "Malayalam", "Hindi_English", "Hindi_Japanese", "English_Japanese"]
Custom_FFMPEG = "-map 0:v -map 0:v:0 -map 0:a:m:language:hin -map 0:s? -c copy"  #@param {type: "string"}
OUTPUT_PATH = ""  #@param {type: "string"}
MAX_FileSize_MB = 0  #@param {type: "number"}
USE_Splitting = False  #@param {type: "boolean"}
Overwrite_OUTPUT = False  #@param {type: "boolean"}
JUST_DOWNLOAD = False  #@param {type: "boolean"}
JUST_EXTRACT = False  #@param {type: "boolean"}

Metadata = Metadata if Metadata else '@MKVmerge'

from IPython.display import clear_output
from IPython.display import HTML
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
# drive.mount('/content/drive')
clear_output()


def printColor(txt, color, htmlTag=''):
    if not htmlTag:
        print(f"\n\033[{color}m{txt}\033[0m")
        return

    html_output = f"""
    <{htmlTag} onclick="copyToClipboard('{txt}')" style='color:{color};'>{txt}
    </{htmlTag}>
    """

    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """
    display(HTML(html_output))

def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator):
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n")
    return finalList

def extract_file_info(file_path):
    file_path = os.path.abspath(file_path)
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def modifyFileName(prefix="", file_path="", suffix=""):
    if not file_path:
        return ""
    file_path = os.path.abspath(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{prefix}{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def getMKVFilesList(folder_path, relative=0):
    folder_path = os.path.abspath(folder_path)
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    Archive_File = os.path.abspath(Archive_File)
    if not extracted_folder:
        extracted_folder = extract_file_info(Archive_File)[2]
    if os.path.exists(extracted_folder):
        printColor(f"Archive File Already Extracted:\n{extracted_folder}\nPlease Delete it if you want to proceed", "#00FF02", "h3")
        return extracted_folder

    os.makedirs(extracted_folder, exist_ok=True)
    printColor(f"Extracting archive to: {extracted_folder}", 92)
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    printColor(f"Extraction complete: {extracted_folder}", "#00FF02", "h3")
    return extracted_folder


def runCommand(cmd, showLog=0):
    printColor(f"Running command: {cmd}", 92)
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()

    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    printColor(f"Status: {process.returncode}", 92)
    return process


def Aria2Downloader(url, output_dir=''):
    printColor(f"Downloading ==> {url}", 92)
    Aria2 = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    if url.startswith("magnet") or url.endswith(".torrent"):
        Aria2 = f"aria2c --bt-enable-lpd=true --bt-save-metadata=true --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    printColor(f"Running command: {Aria2}", 92)

    output_file_path = ""
    process = subprocess.Popen(Aria2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in iter(process.stdout.readline, b''):
        txt = line.decode("utf-8")
        print(txt, end="")
        if txt.find("Download complete: ") > -1:
            output_file_path = txt.split("Download complete: ")
            output_file_path = output_file_path[1].split("\n")[0]
    process.stdout.close()
    process.wait()

    if output_file_path:
        output_file_path = os.path.abspath(output_file_path)
        printColor(f"\nDownload Success ==> {output_file_path}", 92)  # Green color
        return output_file_path
    else:
        printColor(f"Download Error Status code({process.returncode}) ==> {url}", 91)  # Red color
    return url


def getLanguage():
    switch_dict = {
        "All": "-map 0 -map 0:v:0 -c copy",
        "Custom": Custom_FFMPEG,
        "Hindi": "-map 0:v -map 0:v:0 -map 0:a:m:language:hin -map 0:s? -c copy",
        "English": "-map 0:v -map 0:v:0 -map 0:a:m:language:eng -map 0:s? -c copy",
        "Japanese": "-map 0:v -map 0:v:0 -map 0:a:m:language:jpn -map 0:s? -c copy",
        "Tamil": "-map 0:v -map 0:v:0 -map 0:a:m:language:jpn -tam 0:s? -c copy",
        "Telugu": "-map 0:v -map 0:v:0 -map 0:a:m:language:jpn -tel 0:s? -c copy",
        "Malayalam": "-map 0:v -map 0:v:0 -map 0:a:m:language:mal -map 0:s? -c copy",
        "Hindi_English": "-map 0:v -map 0:v:0 -map 0:a:m:language:hin -map 0:a:m:language:eng -map 0:s? -c copy",
        "Hindi_Japanese": "-map 0:v -map 0:v:0 -map 0:a:m:language:hin -map 0:a:m:language:jpn -map 0:s? -c copy",
        "English_Japanese": "-map 0:v -map 0:v:0 -map 0:a:m:language:eng -map 0:a:m:language:jpn -map 0:s? -c copy"
    }
    return switch_dict.get(LANGUAGE, "Invalid_Lang")  # Default case

def getFFmpegOptions():
    FFMPEG_OPTIONS = ""
    FFMPEG_OPTIONS += f" {getLanguage()} -metadata auther=@hamzamusht550 -metadata title=\"{Metadata}\" -metadata:s:v title=\"{Metadata}\" -metadata:s:a title=\"{Metadata}\" -metadata:s:s title=\"{Metadata}\" -ignore_unknown"
    FFMPEG_OPTIONS += " -y" if Overwrite_OUTPUT else ""
    return FFMPEG_OPTIONS

def create_concat_list(filepaths, output_file="list.txt"):
    def sanitize_name(name):
        # Characters to replace
        special_chars = ',:;"!\'?*'
        # Replace each special character with '_'
        for char in special_chars:
            name = name.replace(char, '_')
        return name

    def sanitize_path(path):
        # Split the path into components
        folders = []
        while True:
            path, folder = os.path.split(path)
            if folder:
                folders.append(folder)
            else:
                if path:
                    folders.append(path)
                break
        # Reverse to get the correct order (from root to leaf)
        folders.reverse()
        # Sanitize each folder name
        sanitized_folders = [sanitize_name(folder) for folder in folders]
        # Reconstruct the sanitized path
        sanitized_path = os.path.join(*sanitized_folders)
        return sanitized_path

    def rename_folders_and_files(path):
        # Sanitize the entire path
        sanitized_path = sanitize_path(path)
        # Rename folders and files if the sanitized path is different
        if sanitized_path != path:
            os.renames(path, sanitized_path)
        return sanitized_path

    with open(output_file, "w") as list_file:
        for filepath in filepaths:
            filepath = os.path.abspath(filepath)
            # Sanitize the entire path (including folders)
            sanitized_filepath = rename_folders_and_files(filepath)
            # Write the sanitized filepath to list.txt
            list_file.write(f"file '{sanitized_filepath}'\n")
    return output_file

def mergeMKVs(video_paths, output_dir):
    printColor(f"Merge MKVs ==>\n{video_paths}", "#00FF02", "h4")
    if JUST_DOWNLOAD:
        video_paths_Str = '\n'.join(video_paths)
        printColor("You have enabled JUST_DOWNLOAD\nVideos to Merge:\n{video_paths_Str}", "#00FFFA", "h3")
        return

    video_paths.sort(key=str.lower)
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    os.makedirs(output_dir, exist_ok=True)
    base_name = extract_file_info(video_paths[0])[2] + '.mkv'
    base_output_path = os.path.join(output_dir, base_name)

    # Create list.txt in the output directory
    list_path = os.path.join(output_dir, f"{os.path.splitext(base_name)}.txt")
    create_concat_list(video_paths, list_path)

    temporary_file = modifyFileName("temp_", os.path.split(base_output_path)[1])
    command = f"ffmpeg -f concat -safe 0 -i \"{list_path}\" {getFFmpegOptions()} {temporary_file}\""
    result = runCommand(command, 1)

    if result.returncode == 0:
        suffix = "_merged"
        strSplit = ""
        if USE_Splitting and MAX_FileSize_MB > 999:
            suffix= "_%03d"
            strSplit = f"--split size:{MAX_FileSize_MB}M"
        base_output_path = modifyFileName("", base_output_path, suffix)
        command = f"mkvmerge -o \"{base_output_path}\" {strSplit} \"{temporary_file}\""
        result = runCommand(command, 1)

        if result.returncode == 0:
            printColor(f"Merging Successfull (MKVmerge) ==> {base_output_path}", "#00FF02", "h3")
            return base_output_path
        else:
            printColor(f"Merging Successfull (FFmpeg) ==> {temporary_file}", "#00FFFA", "h3")
            return temporary_file


def singleMKV(video, output_dir=''):
    printColor(f"Single MKV ==> {video}", "#00FF02", "h4")
    if JUST_DOWNLOAD:
        printColor("You have enabled JUST_DOWNLOAD\nSingle Video ==> {video}", "#00FFFA", "h3")
        return

    video = os.path.abspath(video)
    base_output_path = ""
    if not output_dir == "/content/drive/MyDrive" and os.path.exists(output_dir) and os.path.isdir(output_dir):
        base_output_path = os.path.join(output_dir, f"{extract_file_info(video)[2]} ({LANGUAGE}).mkv")
    else:
        base_output_path = modifyFileName("", video, f" ({LANGUAGE})")

    trim = f"-ss {trim_start} -to {trim_end}" if trim_start else ""
    temporary_file = modifyFileName("temp_", os.path.split(base_output_path)[1])
    command = f"ffmpeg -i \"{video}\" {trim} {getFFmpegOptions()} \"{temporary_file}\""
    result = runCommand(command, 1)

    if result.returncode == 0:
        suffix = ""
        strSplit = ""
        if USE_Splitting and MAX_FileSize_MB > 999:
            suffix= "_%03d"
            strSplit = f"--split size:{MAX_FileSize_MB}M"
        base_output_path = modifyFileName("", base_output_path, suffix)
        command = f"mkvmerge -o \"{base_output_path}\" {strSplit} \"{temporary_file}\""
        result = runCommand(command, 1)

        if result.returncode == 0:
            printColor(f"Video Proccessed ==> {base_output_path}", "#00FF02", "h3")
            return base_output_path
        else:
            printColor(f"Video Proccessed (FFmpeg) ==> {temporary_file}", "#00FFFA", "h3")
            return temporary_file


def main(filesListStr, output_dir):
    if not output_dir:
        output_dir = '/content/drive/MyDrive'
    elif not os.path.exists(filesListStr):
        printColor(f"Output Path not exists ==> {output_dir}", "red", "h3")
        return

    valid_Files_List = []
    if os.path.exists(filesListStr):
        printColor(f"MKV_FILES str Exists ==> {filesListStr}", "#00FF02", "h3")
        if filesListStr.endswith('.torrent'): # Type Torrent
            filesListStr = Aria2Downloader(filesListStr)
        elif patoolib.is_archive(filesListStr): # Type Archive
            _Folder = extractArchive(filesListStr)
            if JUST_EXTRACT:
                printColor("You have enabled JUST_EXTRACT.\n{_Folder}", "#00FF02", "h3")
                return
            mergeMKVs(getMKVFilesList(_Folder), output_dir)
        elif os.path.isdir(filesListStr): # Type Folder
            mergeMKVs(getMKVFilesList(filesListStr), output_dir)
        elif filesListStr.endswith('.mkv'): # Type MKV
            singleMKV(filesListStr, output_dir)
        return

    Files_List = strToList(filesListStr, "|")
    _File = Files_List[0]

    if len(Files_List) == 0:
        printColor("Give At Least 1 File", "red", "h3")
        return
    elif len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith(('http', 'magnet:')) or _File.endswith('.torrent'):
            _File = Aria2Downloader(_File)
            if os.path.exists(_File):
                if patoolib.is_archive(_File):
                    _Folder = extractArchive(_File)
                    if JUST_EXTRACT:
                        printColor("You have enabled JUST_EXTRACT.\n{_Folder}", "#00FF02", "h3")
                        return
                    mergeMKVs(getMKVFilesList(_Folder), output_dir)
                elif _File.endswith('.mkv'):
                    singleMKV(_File, output_dir)
        return

    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith(('http', 'magnet:')) or valid_File.endswith('.torrent'):
                valid_File = Aria2Downloader(valid_File, "download_" + extract_file_info(Files_List[0])[2] )
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    printColor(f"File Exists but Unsupported File Format: {valid_File}", "red", "h3")
            else:
                printColor(f"Error FILES {i+1}: {valid_File}", "red", "h3")
                return
        valid_Files_Str = '\n'.join(valid_Files_List)
        printColor(f"Valid Files are:\n{valid_Files_Str}", 92)

    if len(valid_Files_List) > 1:
        mergeMKVs(valid_Files_List, output_dir)

if USE_Splitting and MAX_FileSize_MB <= 999:
    printColor(f"Split Size can't be Smaller than 1000MB", "red", "h2")
else:
    main(MKV_FILES, OUTPUT_PATH)

In [ ]:

#@title Show MKV tracks

FILES_Folder = ""  #@param {type: "string"}

from IPython.display import HTML
from pymediainfo import MediaInfo
import os

def get_track_info(file_path):
    try:
        media_info = MediaInfo.parse(file_path)
        tracks_info = []
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                current_track = [
                track.track_type or "",
                track.codec_id or "",
                track.language or ""
                ]
                current_track_info = f"{track.track_id}: "
                for e in current_track:
                    current_track_info += f"{e} | " if e else ''
                tracks_info.append(current_track_info[:-3])
        tracks_info = '\n'.join(tracks_info)
        #print(tracks_info)
        return tracks_info
    except Exception as e:
        print(f"\nError occurred:\n{e}\n")
        return []

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files


def main(folder_path, use_html=True):
    files = getMKVFilesList(folder_path)
    if not files:
        print("No MKV files found in the specified folder.")
        return

    reference_track_info = get_track_info(files[0])
    # HTML output with copy-to-clipboard buttons
    html_output = f"""
    <h3 onclick="copyToClipboard('{files[0]}')">
    Reference file ==> <b>{files[0]}</b>
    </h3><b>{reference_track_info}</b>
    """

    for i, _File in enumerate(files[1:], start=1):
        current_file_info = get_track_info(_File)
        if reference_track_info == current_file_info:
            html_output += f"""
            <h3 onclick="copyToClipboard('{_File}')" style='color:#00FF02;'>
            {i+1} ==> {_File} (Same Tracks)
            </h3>
            """
        else:
            html_output += f"""
            <h3 onclick="copyToClipboard('{_File}')" style='color:red;'>
            {i+1} ==> {_File} (Different Tracks)
            </h3>
            """
        html_output += f"<b>{current_file_info}</b>"

    # Add JavaScript for copying to clipboard
    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """

    display(HTML(html_output))


main(FILES_Folder)

In [ ]:

#@title URL Downloader

URLs = ""  # @param {type: "string"}
OUTPUT_DIR = "/content"  #@param {type: "string"}
Select_Downloader = "Aria2"  #@param ["Aria2", "wget", "Aria2os"]

import subprocess

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList


def main(urlStr):
    URLs_List = strToList(urlStr, ",")
    for i, _url in enumerate(URLs_List):
        print(f"\n{i+1} ==> {_url}")
        match Select_Downloader:
            case "Aria2":
                !aria2c --continue=true --file-allocation=none -x 16 -s 16 -d "{OUTPUT_DIR}" "{_url}"
            case "wget":
                !wget -P "{OUTPUT_DIR}" "{_url}"
            case "Aria2os":
                cmd = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{OUTPUT_DIR}\" \"{_url}\""
                process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                file_path = process.stdout.decode('utf-8')
                file_path = file_path.split("Download complete: ")
                file_path = file_path[1].split("\n")[0]
                print(os.path.exists(file_path))
                print(file_path)


main(URLs)

In [ ]:
#@title File Delete

Delete_File = ""  #@param {type: "string"}
!ls -lh "{Delete_File}"
!rm -rf "{Delete_File}"

In [ ]:

#@title Save Public Google Drive Files

from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and initialize
auth.authenticate_user()

# Get credentials
credentials = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=credentials)

public_drive_urls = ""  # @param {type: "string"}
# https://drive.google.com/file/d/13Qdkvg8uKGuZv2_8Z1kyUzCMSSlblEEb/view?usp=drivesdk
# https://drive.google.com/drive/folders/1NrBvFvL-Jro3VQBSQhHiOj4-oEEnCIUl
# https://drive.google.com/drive/folders/1glEPV-ISGauk8_HXQ6dV6AfH1HJRsVc6?usp=sharing


def strToList(listStr, separator):
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n")
    return finalList

def extractPublicDriveID(url):
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    try:
        # Check if the URL is a file or folder and extract the ID accordingly
        file_id = ""
        if "file/d/" in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif "folders/" in url:
            file_id = url.split('folders/')[1].split('/')[0]
        else:
            return ""  # Invalid URL format

        if "?usp=sharing" in file_id:
            file_id = file_id.replace("?usp=sharing","")

        # Create a file object and load metadata using the extracted ID
        file = drive.CreateFile({'id': file_id})
        file.FetchMetadata()  # Fetch metadata to verify ID and access permissions
        return file['id']
    except Exception as e:
        return ""

def getDriveMetadata(_id, metadata_key):
    _metadata = service.files().get(fileId=_id, fields=metadata_key).execute()
    metadata_value = _metadata.get(metadata_key)
    return metadata_value

# Function to create a folder in your Drive
def create_drive_folder(name, parent_id=None):
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id] if parent_id else ["root"]
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

# Function to copy a drive file
def transfer_drive_file(file_id, name="", parent_id=None):
    if not name:
        name = getDriveMetadata(file_id, 'name')
    copied_file_metadata = {
        'name': name,
        'parents': [parent_id] if parent_id else ["root"]
    }
    copied_file = service.files().copy(fileId=file_id, body=copied_file_metadata).execute()
    # return name, copied_file.get('id')
    return name

# Function to recursively copy a folder
def transfer_drive_folder(folder_id, parent_id=None):
    # Get the folder metadata
    folder_name = getDriveMetadata(folder_id, 'name')

    # Create the folder in your Drive
    new_folder_id = create_drive_folder(folder_name, parent_id)

    # List all items in the folder
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()

    items = results.get('files', [])

    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively copy subfolder
            transfer_drive_folder(item['id'], new_folder_id)
        else:
            # Copy individual file
            transfer_drive_file(item['id'], item['name'], new_folder_id)

    print(f"Copied folder '{folder_name}' to your Drive (ID: {new_folder_id})")


def downloadFromPublicDrive(Drive_URLs):
    colab_root = "/content/"
    drive_root = "/content/drive/MyDrive/"
    file_folder_name = ""
    file_folder_path = ""

    Drive_URLs_List = strToList(Drive_URLs, "|")
    for url in Drive_URLs_List:
        public_drive_id = extractPublicDriveID(url)
        print('ID: ', public_drive_id)
        if "file/d/" in url:
            # Start copying the folder
            try:
                file_folder_name = transfer_drive_file(public_drive_id)
                file_folder_path = drive_root + file_folder_name
                print(f"Drive 🗄️ Transfer Successfull: {file_folder_name}")
            except HttpError as error:
                print(f"Drive error occurred: {error}")
        elif "folders/" in url:
            # Start copying the folder
            try:
                transfer_drive_folder(public_drive_id)
                file_folder_name = getDriveMetadata(public_drive_id, 'name')
                file_folder_path = drive_root + file_folder_name
                print(f"Drive 📂 Transfer Successfull: {file_folder_name}")
            except HttpError as error:
                print(f"Drive error occurred: {error}")


downloadFromPublicDrive(public_drive_urls)


# _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.

In [ ]:


# !pip install pyrogram tgcrypto nest_asyncio

#@title Telegram Uploader
MyFile = ""  #@param {type: "string"}
Thumbnail_Path = ""  #@param {type: "string"}
ActionType = "video"  #@param ["video", "doc", "pic", "audio", "msg"]
TXT = "Here's My Captions"  #@param {type: "string"}

''' Use Following links to get Telegram ids & apis
https://api.telegram.org/bot7080297328:AAFTbHgHaC1Nb0FG9z_Z-f9tCiPAcAEwevk/deleteWebhook
https://api.telegram.org/bot7080297328:AAFTbHgHaC1Nb0FG9z_Z-f9tCiPAcAEwevk/getUpdates
https://my.telegram.org/
'''

import ffmpeg
import time
import random
import string
import os
import subprocess
import json
import asyncio
import nest_asyncio
from pymediainfo import MediaInfo
from pyrogram import Client
from pyrogram import enums

Thumbnail = Thumbnail_Path if os.path.exists(Thumbnail_Path) else ''

# Apply nested event loop patch
nest_asyncio.apply()

def getRandomText(length):
    random_text = ''.join(random.choices(string.ascii_letters, k=length))
    return random_text

def get_track_info(file_path):
    try:
        media_info = MediaInfo.parse(file_path)
        tracks_info = []
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                current_track = [
                track.track_type or "",
                track.codec_id or "",
                track.language or ""
                ]
                current_track_info = f"{track.track_id}: "
                for e in current_track:
                    current_track_info += f"{e} | " if e else ''
                tracks_info.append(current_track_info[:-3])
        tracks_info = '\n'.join(tracks_info)
        print(tracks_info)
        return tracks_info
    except Exception as e:
        print(f"\nError occurred:\n{e}\n")
        return []


def getDurationSec(file_path):
    probe = ffmpeg.probe(file_path)
    duration = float(probe['format']['duration'])
    duration_seconds = int(duration)
    return duration_seconds

def format_size(bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
        if bytes < 1024:
            return f"{bytes:.2f}{unit}"
        bytes /= 1024

async def tg_upload_progress(current, total, *args):
    _progress = (f"{format_size(current)} / {format_size(total)} ({current / total * 100:.2f}%)")
    # await main("msg", _progress)
    print(_progress)

async def telegram_utility(action, client, chat_id, **kwargs):
    """
    Perform various Telegram actions using Pyrogram.

    Parameters:
    - action (str): The action to perform. Options: 'send_message', 'send_document', 'send_photo', 'send_video'.
    - client (Client): An active Pyrogram client.
    - chat_id (int/str): The target chat ID or username.
    - **kwargs: Additional parameters for specific actions.
    """
    if action == "send_message":
        # Send a message
        await client.send_message(chat_id=chat_id, text=kwargs.get("text", ""))
    elif action == "send_document":
        # Send a document/file
        await client.send_document(
            chat_id=chat_id,
            document=kwargs.get("document"),
            caption=kwargs.get("caption", ""),
            parse_mode=kwargs.get("parse_mode", None),
            progress=tg_upload_progress
        )
    elif action == "send_photo":
        # Send a photo
        await client.send_photo(
            chat_id=chat_id,
            photo=kwargs.get("photo"),
            caption=kwargs.get("caption", ""),
            parse_mode=kwargs.get("parse_mode", None),
            progress=tg_upload_progress
        )
    elif action == "send_audio":
        # Send an audio file
        await client.send_audio(
            chat_id=chat_id,
            audio=kwargs.get("audio"),
            caption=kwargs.get("caption", ""),
            parse_mode=kwargs.get("parse_mode", None),
            progress=tg_upload_progress
        )
    elif action == "send_video":
        # Send a video
        await client.send_video(
            chat_id=chat_id,
            video=kwargs.get("video"),
            caption=kwargs.get("caption", ""),
            parse_mode=kwargs.get("parse_mode", None),
            duration=kwargs.get("duration", None),  # Optional: video duration
            width=kwargs.get("width", None),  # Optional: video width
            height=kwargs.get("height", None),  # Optional: video height
            thumb=kwargs.get("thumb", None),  # Optional: thumbnail image
            progress=tg_upload_progress
        )
    else:
        raise ValueError(f"Unsupported action: {action}")


async def main(action_type, txt='', file_path=''):
    if action_type != "msg":
        if not os.path.exists(file_path):
            print(f"File not Exists: {file_path}")
            return

    # Create Pyrogram client
    bot_token = "BOT_TOKEN"
    chat_id = "@USERNAME" # Replace with profile ID or username
    api_id = CHAT_ID  # Replace with your Telegram API ID
    api_hash = "API_HASH"

    action_type = action_type.strip().lower()
    async with Client(getRandomText(6), bot_token=bot_token, api_id=api_id, api_hash=api_hash) as app:
        app.set_parse_mode(enums.ParseMode.MARKDOWN)
        start_time = time.time()
        match action_type:
            case "msg":
                # Send a message
                await telegram_utility("send_message", app, chat_id, text=txt)

            case "doc":
                # Send a document
                await telegram_utility("send_document", app, chat_id, document=file_path, caption=txt)

            case "pic":
                # Send a photo
                await telegram_utility("send_photo", app, chat_id, photo=file_path, caption=txt)

            case "audio":
                # Send a photo
                await telegram_utility("send_audio", app, chat_id, audio=file_path, caption=txt)

            case "video":
                if not file_path.endswith('.mkv'):
                    print(f"Not MKV File: {file_path}")
                    return

                # Send a video
                global Thumbnail
                if not Thumbnail:
                    Thumbnail = "thumb.jpg"
                    !rm -rf "{_thumb}"
                    !ffmpeg -i "{file_path}" -vf "select=gt(scene\,0.1),scale=1280:720" -vsync vfr -frames:v 1 -y "{Thumbnail}"

                _duration = getDurationSec(file_path)
                track = get_track_info(file_path)
                txt = f"**🎬 {os.path.split(file_path)[1]}**\n```\n{track}\n```\n{txt}"
                await telegram_utility("send_video", app, chat_id, video=file_path, caption=txt,
                    duration=_duration,  # Optional: video duration
                    width=1280,  # Optional: video width
                    height=720,  # Optional: video height
                    thumb=Thumbnail
                )

    elapsed_time = time.time() - start_time
    print(f"Action {action_type} Completed: {elapsed_time:.2f}s")


# Run the async function
await main(ActionType, TXT, MyFile)